In [76]:
#imports
import pandas as pd
import numpy as np

from math import pi
from statistics import mean
from bokeh.transform import cumsum
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from collections import Counter
import plotly.graph_objects as go
import plotly.express as px

from fractions import Fraction
concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM']

#data gathering
notes = pd.read_excel('../data/all_sections_notes.xlsx')
student_data = pd.read_excel('../data/student_information_filtered.xlsx')

## Data Gathering

In [77]:
student_data = student_data[student_data['NUMERO'].notna()]
student_data.drop(columns={'EPS', 'Comentarios','Falta por consentimiento'}, axis=1, inplace=True)
student_data['DOCUMENTO '] = pd.to_numeric(student_data['DOCUMENTO '])
student_data.columns

#We rename the columns to make them easier to work with
notes.rename(columns={
    #WEEK 1
    'OrgDefinedId':'user',
    'Secciones':'sec',
    'Entrega de la mañana D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1AM',
    'Entrega de la tarde D1 Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'1PM',
    'Entrega de la mañana (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2AM',
    'Entrega de la tarde (D2) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'2PM',
    'Entrega de la mañana (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3AM',
    'Entrega de la tarde (D3) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'3PM',
    'Entrega de la mañana (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4AM',
    'Entrega de la tarde (D4) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 1\Week1>':'4PM',

    'Semana 1\Week1 Subtotal Numerator':'week_1_numerator',
    'Semana 1\Week1 Subtotal Denominator':'week_1_denominator',
    
    #WEEK 2
    'Entrega de la tarde (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5PM',
    'Entrega de la mañana (D5) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'5AM',
    'Entrega de la mañana (D6) Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>':'6AM',

    'Semana 2\Week 2 Subtotal Numerator':'week_2_numerator',
    'Semana 2\Week 2 Subtotal Denominator':'week_2_denominator',

    'Calculated Final Grade Numerator':'final_numerator',
    'Calculated Final Grade Denominator':'final_denominator'
}, inplace = True)

#we change the type to a numeric float64 from 'object'
#Week 1
notes['1AM'] = pd.to_numeric(notes['1AM'])
notes['1PM'] = pd.to_numeric(notes['1PM'])
notes['2AM'] = pd.to_numeric(notes['2AM'])
notes['2PM'] = pd.to_numeric(notes['2PM'])
notes['3AM'] = pd.to_numeric(notes['3AM'])
notes['3PM'] = pd.to_numeric(notes['3PM'])
notes['4AM'] = pd.to_numeric(notes['4AM'])
notes['4PM'] = pd.to_numeric(notes['4PM'])
#Week 2
notes['5AM'] = pd.to_numeric(notes['5AM'])
notes['5PM'] = pd.to_numeric(notes['5PM'])
notes['6AM'] = pd.to_numeric(notes['6AM'])



notes['sec'].replace(['Sección No. 1', 'Sección No. 2', 'Sección No. 3','Sección No. 4','Sección No. 5','Sección No. 6','Sección No. 7',
'Sección No. 8','Sección No. 9','Sección No. 10 - Las morcillitas'], [1,2,3,4,5,6,7,8,9,10], inplace=True)

notes.fillna(0.0, inplace=True)
notes['user'] = pd.to_numeric(notes['user'])

notes.drop(columns={'Adjusted Final Grade Numerator', 'Adjusted Final Grade Denominator', 'Username'}, axis=1, inplace=True)

data = pd.merge(notes, student_data, how='inner', left_on='user', right_on='DOCUMENTO ')

data.drop(columns={'EMAIL ESTUDIANTE',
       'CORREO ACUDIENTE', 'ESPAÑOL', 'MATEMATICAS', 'PROMEDIO NOTAS',
       'ESCALA', 'PUNTAJE', 'RESULTADO', 'EMAIL CONFIRMACION ','CSBRIDGE SEC', 'Email','DOCUMENTO '}, axis=1, inplace=True)

#we recalculate the numerator and denominator for every student, as the data didn't take NaN into account.
for index in data.index:
        #week 1
        a = (sum([data.loc[index, '1AM'],
        data.loc[index, '1PM'],
        data.loc[index, '2AM'],
        data.loc[index, '2PM'],
        data.loc[index, '3AM'],
        data.loc[index, '3PM'],
        data.loc[index, '4AM'],
        data.loc[index, '4PM']]))
        b = 8.0

        data.loc[index, 'week_1_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'week_1_denominator'] = Fraction(a/b).denominator

        #week 2
        a = (sum([data.loc[index, '5AM'],
                data.loc[index, '5PM'],
                data.loc[index, '6AM']]))
        b = 3.0

        data.loc[index, 'week_2_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'week_2_denominator'] = Fraction(a/b).denominator

        #final values, this operation might come as redundant at first, but it is quite important for final averages. Either way, i might 
        #optimize it in the future.
        a = (sum([data.loc[index, '1AM'],
        data.loc[index, '1PM'],
        data.loc[index, '2AM'],
        data.loc[index, '2PM'],
        data.loc[index, '3AM'],
        data.loc[index, '3PM'],
        data.loc[index, '4AM'],
        data.loc[index, '4PM'],
        data.loc[index, '5AM'],
        data.loc[index, '5PM'],
        data.loc[index, '6AM']]))
        b = 11.0
        
        data.loc[index, 'final_numerator'] = Fraction(a/b).numerator
        data.loc[index, 'final_denominator'] = Fraction(a/b).denominator

data.columns


Index(['user', 'Last Name', 'First Name', 'sec', '1AM', '1PM', '2AM', '2PM',
       '3AM', '3PM', '4AM', '4PM', 'week_1_numerator', 'week_1_denominator',
       '5PM', '5AM', '6AM',
       'Entrega de la Final del curso Points Grade <Numérico Puntos máx.:5 Categoría:Semana 2\Week 2>',
       'week_2_numerator', 'week_2_denominator', 'final_numerator',
       'final_denominator', 'NUMERO', 'ESTUDIANTE', 'Asistió', 'GENERO',
       'COLEGIO', 'GRADO'],
      dtype='object')

# Five-Zero Majority

In [78]:
def five_zero_majority(dat):
    five_majority = []
    zero_majority = []
    for index in dat.index:
        a = [dat.loc[index, '1AM'],
        dat.loc[index, '1PM'],
        dat.loc[index, '2AM'],
        dat.loc[index, '2PM'],
        dat.loc[index, '3AM'],
        dat.loc[index, '3PM'],
        dat.loc[index, '4AM'],
        dat.loc[index, '4PM'],
        dat.loc[index, '5AM'],
        dat.loc[index, '5PM'],
        dat.loc[index, '6AM']]
        ln_a_half = (len(a)//2)+1
        cnt_a = Counter(a)

        if Counter(a)[5.0] >= ln_a_half:
            five_majority.append(data.iloc[a])
        if Counter(a)[0.0] >= ln_a_half:
            zero_majority.append(data.iloc[a])
    print((len(five_majority)/len(data.index))*100)
    print((len(zero_majority)/len(data.index))*100)
    figure = go.Figure(
        data = [go.Bar
        (y= [(len(five_majority)/len(data.index))*100, ((len(zero_majority)/len(data.index))*100)],        
        x= ["Five Majority", "Zero Majority"]),
        ],
    )
    figure.update_layout(yaxis_range=[0,100])
    figure.show()
five_zero_majority(data)

65.65656565656566
18.181818181818183


# Five Zero Majority: Gender Discriminated

In [79]:
def five_zero_majority_male(dat):
    dat = dat.query("GENERO == 'H'")
    five_majority = []
    zero_majority = []
    for index in dat.index:
        a = [dat.loc[index, '1AM'],
        dat.loc[index, '1PM'],
        dat.loc[index, '2AM'],
        dat.loc[index, '2PM'],
        dat.loc[index, '3AM'],
        dat.loc[index, '3PM'],
        dat.loc[index, '4AM'],
        dat.loc[index, '4PM'],
        dat.loc[index, '5AM'],
        dat.loc[index, '5PM'],
        dat.loc[index, '6AM']]
        ln_a_half = (len(a)//2)+1
        cnt_a = Counter(a)

        if Counter(a)[5.0] >= ln_a_half:
            five_majority.append(data.iloc[a])
        if Counter(a)[0.0] >= ln_a_half:
            zero_majority.append(data.iloc[a])
    
    print("Five Majority on male students: " + str((len(five_majority)/len(dat.index))*100))
    print("Zero Majority on male students: " + str((len(zero_majority)/len(dat.index))*100))
    figure = go.Figure(
        data = [go.Bar
        (y= [(len(five_majority)/len(dat.index))*100, ((len(zero_majority)/len(dat.index))*100)],        
        x= ["Five Majority", "Zero Majority"]),
        ],
    )
    figure.update_layout(yaxis_range=[0,100])
    figure.show()

five_zero_majority_male(data)

def five_zero_majority_female(dat):
    dat = dat.query("GENERO == 'M'")
    five_majority = []
    zero_majority = []
    for index in dat.index:
        a = [dat.loc[index, '1AM'],
        dat.loc[index, '1PM'],
        dat.loc[index, '2AM'],
        dat.loc[index, '2PM'],
        dat.loc[index, '3AM'],
        dat.loc[index, '3PM'],
        dat.loc[index, '4AM'],
        dat.loc[index, '4PM'],
        dat.loc[index, '5AM'],
        dat.loc[index, '5PM'],
        dat.loc[index, '6AM']]
        ln_a_half = (len(a)//2)+1
        cnt_a = Counter(a)

        if Counter(a)[5.0] >= ln_a_half:
            five_majority.append(data.iloc[a])
        if Counter(a)[0.0] >= ln_a_half:
            zero_majority.append(data.iloc[a])
    print("Five Majority on female students: " + str((len(five_majority)/len(dat.index))*100))
    print("Zero Majority on female students: " + str((len(zero_majority)/len(dat.index))*100))
    figure = go.Figure(
        data = [go.Bar
        (y= [(len(five_majority)/len(dat.index))*100, ((len(zero_majority)/len(dat.index))*100)],        
        x= ["Five Majority", "Zero Majority"]),
        ],
    )
    figure.update_layout(yaxis_range=[0,100])
    figure.show()

five_zero_majority_female(data)

Five Majority on male students: 61.224489795918366
Zero Majority on male students: 24.489795918367346


Five Majority on female students: 70.0
Zero Majority on female students: 12.0


# Pie graph: Gender Discriminated.

In [80]:

concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM'] 

def note_dist_male(dat):
    note_ranges = {
    '5: Perfect implementation.':0,
    '3.5 - 4.9: Functional implementation with small problems.':0,
    '0.1 - 3.4: Implementation with serious problems (not working)':0,
    "0: Did not submit the work.": 0
    }

    for index in dat.index:
        for concept in concept_checks:
            grade = data.loc[index, concept]
            if grade == 0:
                note_ranges["0: Did not submit the work."] += 1
            elif grade == 5:
                note_ranges['5: Perfect implementation.'] += 1
            elif grade in np.arange(3.5, 5):
                note_ranges['3.5 - 4.9: Functional implementation with small problems.'] += 1
            else:
                note_ranges['0.1 - 3.4: Implementation with serious problems (not working)'] += 1
    print("--RANGES FOR MALE STUDENTS--")
    print('5: Perfect implementation.- '+
        str((note_ranges['5: Perfect implementation.']/ sum(note_ranges.values()))*100)
        +"%")
    print('3.5 - 4.9: Functional implementation with small problems.- '+
        str((note_ranges['3.5 - 4.9: Functional implementation with small problems.']/ sum(note_ranges.values()))*100)
        +"%")
    print('0.1 - 3.4: Implementation with serious problems (not working).- '+
        str((note_ranges['0.1 - 3.4: Implementation with serious problems (not working)']/ sum(note_ranges.values()))*100)
        +"%")
    print('0: Did not submit the work.'+
        str((note_ranges['0: Did not submit the work.']/ sum(note_ranges.values()))*100)
        +"%")
    
note_dist_male(data.query("GENERO == 'H'"))

def note_dist_female(dat):
    note_ranges = {
    '5: Perfect implementation.':0,
    '3.5 - 4.9: Functional implementation with small problems.':0,
    '0.1 - 3.4: Implementation with serious problems (not working)':0,
    "0: Did not submit the work.": 0
    }

    for index in dat.index:
        for concept in concept_checks:
            grade = data.loc[index, concept]
            if grade == 0:
                note_ranges["0: Did not submit the work."] += 1
            elif grade == 5:
                note_ranges['5: Perfect implementation.'] += 1
            elif grade in np.arange(3.5, 5):
                note_ranges['3.5 - 4.9: Functional implementation with small problems.'] += 1
            else:
                note_ranges['0.1 - 3.4: Implementation with serious problems (not working)'] += 1
    print("--RANGES FOR FEMALE STUDENTS--")
    print('5: Perfect implementation.- '+
        str((note_ranges['5: Perfect implementation.']/ sum(note_ranges.values()))*100)
        +"%")
    print('3.5 - 4.9: Functional implementation with small problems.- '+
        str((note_ranges['3.5 - 4.9: Functional implementation with small problems.']/ sum(note_ranges.values()))*100)
        +"%")
    print('0.1 - 3.4: Implementation with serious problems (not working).- '+
        str((note_ranges['0.1 - 3.4: Implementation with serious problems (not working)']/ sum(note_ranges.values()))*100)
        +"%")
    print('0: Did not submit the work.'+
        str((note_ranges['0: Did not submit the work.']/ sum(note_ranges.values()))*100)
        +"%")


note_dist_female(data.query("GENERO == 'M'"))

--RANGES FOR MALE STUDENTS--
5: Perfect implementation.- 54.73098330241187%
3.5 - 4.9: Functional implementation with small problems.- 8.905380333951761%
0.1 - 3.4: Implementation with serious problems (not working).- 3.153988868274583%
0: Did not submit the work.33.209647495361786%
--RANGES FOR FEMALE STUDENTS--
5: Perfect implementation.- 61.63636363636363%
3.5 - 4.9: Functional implementation with small problems.- 7.818181818181818%
0.1 - 3.4: Implementation with serious problems (not working).- 3.8181818181818183%
0: Did not submit the work.26.727272727272727%


# Ranges per School

In [81]:
schools = []
for i in range(len(data)):
    schools.append(data.loc[i, 'COLEGIO'])
schools = list(set(schools))

def ranges_per_school(data):
    for i in schools:

        grades = []

        dat = data.query("COLEGIO == '"+i+"'")
        for index in dat.index:
            grades.append(float(dat.loc[index, 'final_numerator'])/float(dat.loc[index, 'final_denominator']))
        print(i)
        print("No. Estudiantes: "+str(len(dat)))
        print("promedio: "+str(sum(grades)/len(grades)))
        print("")
        
    pass

ranges_per_school(data)

COLEGIO ANGLO COLOMBIANO
No. Estudiantes: 10
promedio: 3.2909090909090906

COLEGIO NAVAL SANTAFÉ DE BOGOTÁ
No. Estudiantes: 8
promedio: 2.5238636363636364

COLEGIO CRISTIANO GRACIA Y AMOR
No. Estudiantes: 5
promedio: 4.054545454545455

COLEGIO CRISTIANO SEMILLA DE VIDA 
No. Estudiantes: 10
promedio: 3.831818181818181

COLEGIO SANTA FRANCISCA ROMANA
No. Estudiantes: 5
promedio: 3.163636363636363

GIMNASIO CAMPESTRE BETH SHALOM 
No. Estudiantes: 4
promedio: 2.9318181818181817

SAN JOSE I E D 
No. Estudiantes: 4
promedio: 3.3181818181818183

GIMNASIO MODERNO
No. Estudiantes: 3
promedio: 4.363636363636363

GIMNASIO CAMPESTRE LOS SAUCES
No. Estudiantes: 5
promedio: 3.447272727272727

PATRIA
No. Estudiantes: 1
promedio: 0.0

COLEGIO HELADIA MEJIA I.E.D.
No. Estudiantes: 14
promedio: 3.3720779220779216

LICEO EL ENCUENTRO
No. Estudiantes: 9
promedio: 3.087878787878788

GIMNASIO CAMPESTRE
No. Estudiantes: 1
promedio: 4.863636363636363

GIMNASIO FEMENINO
No. Estudiantes: 5
promedio: 3.847272727

# Gender Distribution for schools.

In [83]:
schools = []
for i in range(len(data)):
    schools.append(data.loc[i, 'COLEGIO'])
schools = list(set(schools))

def school_gender_dist(data):
    for i in schools:

        dat = data.query("COLEGIO == '"+i+"'")
        dat_male = dat.query("GENERO == 'H'")
        dat_female = dat.query("GENERO == 'M'")
        print("SCHOOL: "+i)
        print("FEMALE PERCENTAGE: "+(str((len(dat_female)/len(dat))*100))+"%")
        print("MALE PERCENTAGE: "+(str((len(dat_male)/len(dat))*100))+"%")


school_gender_dist(data)

SCHOOL: COLEGIO ANGLO COLOMBIANO
FEMALE PERCENTAGE: 40.0%
MALE PERCENTAGE: 60.0%
SCHOOL: COLEGIO NAVAL SANTAFÉ DE BOGOTÁ
FEMALE PERCENTAGE: 25.0%
MALE PERCENTAGE: 75.0%
SCHOOL: COLEGIO CRISTIANO GRACIA Y AMOR
FEMALE PERCENTAGE: 40.0%
MALE PERCENTAGE: 60.0%
SCHOOL: COLEGIO CRISTIANO SEMILLA DE VIDA 
FEMALE PERCENTAGE: 40.0%
MALE PERCENTAGE: 60.0%
SCHOOL: COLEGIO SANTA FRANCISCA ROMANA
FEMALE PERCENTAGE: 100.0%
MALE PERCENTAGE: 0.0%
SCHOOL: GIMNASIO CAMPESTRE BETH SHALOM 
FEMALE PERCENTAGE: 0.0%
MALE PERCENTAGE: 100.0%
SCHOOL: SAN JOSE I E D 
FEMALE PERCENTAGE: 100.0%
MALE PERCENTAGE: 0.0%
SCHOOL: GIMNASIO MODERNO
FEMALE PERCENTAGE: 0.0%
MALE PERCENTAGE: 100.0%
SCHOOL: GIMNASIO CAMPESTRE LOS SAUCES
FEMALE PERCENTAGE: 60.0%
MALE PERCENTAGE: 40.0%
SCHOOL: PATRIA
FEMALE PERCENTAGE: 0.0%
MALE PERCENTAGE: 100.0%
SCHOOL: COLEGIO HELADIA MEJIA I.E.D.
FEMALE PERCENTAGE: 50.0%
MALE PERCENTAGE: 50.0%
SCHOOL: LICEO EL ENCUENTRO
FEMALE PERCENTAGE: 55.55555555555556%
MALE PERCENTAGE: 44.4444444444444

# Submission rates per school

In [85]:
schools = []
for i in range(len(data)):
    schools.append(data.loc[i, 'COLEGIO'])
schools = list(set(schools))

concept_checks = ['1AM','1PM','2AM','2PM','3AM','3PM','4AM','4PM','5AM','5PM','6AM'] 


def school_submission_rate(data):
    for i in schools:
        dat = data.query("COLEGIO == '"+i+"'")
        note_ranges = {
        '5: Perfect implementation.':0,
        '3.5 - 4.9: Functional implementation with small problems.':0,
        '0.1 - 3.4: Implementation with serious problems (not working)':0,
        "0: Did not submit the work.": 0
        }
        for index in dat.index:
            for concept in concept_checks:
                grade = data.loc[index, concept]
                if grade == 0:
                    note_ranges["0: Did not submit the work."] += 1
                elif grade == 5:
                    note_ranges['5: Perfect implementation.'] += 1
                elif grade in np.arange(3.5, 5):
                    note_ranges['3.5 - 4.9: Functional implementation with small problems.'] += 1
                else:
                    note_ranges['0.1 - 3.4: Implementation with serious problems (not working)'] += 1
            print("--RANGES FOR "+ i +"--")
            print('5: Perfect implementation.- '+
                str((note_ranges['5: Perfect implementation.']/ sum(note_ranges.values()))*100)
                +"%")
            print('3.5 - 4.9: Functional implementation with small problems.- '+
                str((note_ranges['3.5 - 4.9: Functional implementation with small problems.']/ sum(note_ranges.values()))*100)
                +"%")
            print('0.1 - 3.4: Implementation with serious problems (not working).- '+
                str((note_ranges['0.1 - 3.4: Implementation with serious problems (not working)']/ sum(note_ranges.values()))*100)
                +"%")
            print('0: Did not submit the work.'+
                str((note_ranges['0: Did not submit the work.']/ sum(note_ranges.values()))*100)
                +"%")       

school_submission_rate(data)          
            

--RANGES FORCOLEGIO ANGLO COLOMBIANO--
5: Perfect implementation.- 9.090909090909092%
3.5 - 4.9: Functional implementation with small problems.- 0.0%
0.1 - 3.4: Implementation with serious problems (not working).- 9.090909090909092%
0: Did not submit the work.81.81818181818183%
--RANGES FORCOLEGIO ANGLO COLOMBIANO--
5: Perfect implementation.- 50.0%
3.5 - 4.9: Functional implementation with small problems.- 0.0%
0.1 - 3.4: Implementation with serious problems (not working).- 4.545454545454546%
0: Did not submit the work.45.45454545454545%
--RANGES FORCOLEGIO ANGLO COLOMBIANO--
5: Perfect implementation.- 42.42424242424242%
3.5 - 4.9: Functional implementation with small problems.- 15.151515151515152%
0.1 - 3.4: Implementation with serious problems (not working).- 3.0303030303030303%
0: Did not submit the work.39.39393939393939%
--RANGES FORCOLEGIO ANGLO COLOMBIANO--
5: Perfect implementation.- 40.909090909090914%
3.5 - 4.9: Functional implementation with small problems.- 13.63636363636